In [2]:
pip install fastapi uvicorn scikit-learn


   ---------------------------------------- 0.0/95.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/95.2 kB ? eta -:--:--
   ---------------------------------- ----- 81.9/95.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 95.2/95.2 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ---------------------------------------- 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.0 kB ? eta -:--:--
   ---------------------------------------- 72.0/72.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel
from typing import Literal
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

app = FastAPI()

        
# Dummy training data
questions = [
    "What is the Pythagorean theorem?",
    "How do plants perform photosynthesis?",
    "Define a noun.",
    "What is the derivative of x squared?",
    "What are Newton's laws?",
    "What is a metaphor?"
]
labels = ["Math", "Science", "English", "Math", "Science", "English"]

# Vectorizer and Model
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

model = MultinomialNB()
model.fit(X, labels)

# ======= Input/Output Schema =======

class QuestionInput(BaseModel):
    question: str

class ClassificationOutput(BaseModel):
    topic: Literal["Math", "Science", "English"]
    confidence: float

# ======= API Endpoint =======

@app.post("/classify-question", response_model=ClassificationOutput)
def classify_question(input: QuestionInput):
    question = input.question.strip()

    if not question:
        raise HTTPException(status_code=400, detail="Question cannot be empty.")

    try:
        X_input = vectorizer.transform([question])
        prediction = model.predict(X_input)[0]
        proba = model.predict_proba(X_input)
        confidence = round(np.max(proba), 2)
        
        return {"topic": prediction, "confidence": confidence}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {str(e)}")
